In [ ]:
import requests
import json
from IPython.display import JSON
from keys import *

###Carga las herramientas esenciales para conectar a internet

In [ ]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

##Establecen dónde buscar los datos (URL del aeropuerto EHAM y rango de tiempo), qué buscar (parámetros para solo llegadas, vuelos de carga, etc.) y cómo autenticarse (las cabeceras con la clave de la API).

In [ ]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

##Comprueba si la conexión fue exitosa, else: print(...): Si hay un fallo, notifica el error.



Data appended to data.json file.


In [ ]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

##carga el JSON a una variable y luego contiene líneas de código que acceden a información específica


In [ ]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

##Este bloque procesa y estructura los datos para el análisis.
##Muestra un resumen de las primeras filas del DataFrame final.

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,CZ 7758,Unknown,China Southern,Boeing 737-800 (winglets),2025-09-01 06:00Z,2025-09-01 08:00+02:00
0,KQ 1364,Unknown,Kenya,Boeing 737-800 (winglets),2025-09-01 06:00Z,2025-09-01 08:00+02:00
0,DL 9274,Unknown,Delta Air Lines,Boeing 737-800 (winglets),2025-09-01 06:00Z,2025-09-01 08:00+02:00
0,UX 3749,Unknown,Air Europa,Boeing 737-800 (winglets),2025-09-01 06:00Z,2025-09-01 08:00+02:00
0,KL 1310,Warsaw,KLM,Boeing 737-800 (winglets),2025-09-01 06:00Z,2025-09-01 08:00+02:00
